In [104]:
import tensorflow as tf
import pandas as pd                                                     # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np                                                      # linear algebra
from tensorflow.keras.models import Sequential                          # model type
from tensorflow.keras.layers import Dense, Dropout, Flatten             # layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D                # layers
from tensorflow.keras.activations import relu, softmax                  # activation functions
from tensorflow.keras.optimizers import SGD                             # optimizer
from tensorflow.keras.losses import sparse_categorical_crossentropy            
from tensorflow.keras.utils import to_categorical                       # one-hot encoding
from tensorflow.keras.datasets import mnist, fashion_mnist              # dataset
from tensorflow.keras.callbacks import EarlyStopping                    # early stopping
from tensorflow.keras.callbacks import ModelCheckpoint                  # model checkpoint
from tensorflow.keras.models import load_model                          # load model
from tensorflow.keras.preprocessing.image import ImageDataGenerator     # data augmentation
import plotly.express as px                                             # plotly

In [29]:
# load data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [30]:
# display image b/w
px.imshow(x_train[1], binary_string=True, height=300, width=300)

In [31]:
input_shape = (28, 28, 1)

In [36]:
# creating the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation=relu, input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation=relu))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation=relu))
model.add(Dropout(0.5))
model.add(Dense(10, activation=softmax))
model.compile(loss=sparse_categorical_crossentropy, optimizer=SGD(), metrics=['accuracy'])
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 5, 5, 64)          0         
                                                                 
 flatten_5 (Flatten)         (None, 1600)             

In [37]:
# train the model
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
ckpt = ModelCheckpoint(filepath='model.h5', monitor='val_loss',
                        save_best_only=True, verbose=1, mode='auto')
batch_size = 128
epochs = 10

In [39]:
history = model.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=1, 
          validation_data=(x_test, y_test),
          callbacks=[es, ckpt],
          shuffle=True)

Epoch 1/10
469/469 [==============================] - ETA: 0s - loss: 0.4944 - accuracy: 0.8200
Epoch 1: val_loss improved from 0.42693 to 0.42166, saving model to model.h5
469/469 [==============================] - 19s 41ms/step - loss: 0.4944 - accuracy: 0.8200 - val_loss: 0.4217 - val_accuracy: 0.8399
Epoch 2/10
468/469 [============================>.] - ETA: 0s - loss: 0.4873 - accuracy: 0.8215
Epoch 2: val_loss improved from 0.42166 to 0.41927, saving model to model.h5
469/469 [==============================] - 19s 40ms/step - loss: 0.4875 - accuracy: 0.8214 - val_loss: 0.4193 - val_accuracy: 0.8406
Epoch 3/10
469/469 [==============================] - ETA: 0s - loss: 0.4786 - accuracy: 0.8252
Epoch 3: val_loss did not improve from 0.41927
469/469 [==============================] - 18s 39ms/step - loss: 0.4786 - accuracy: 0.8252 - val_loss: 0.4253 - val_accuracy: 0.8355
Epoch 4/10
469/469 [==============================] - ETA: 0s - loss: 0.4727 - accuracy: 0.8268
Epoch 4: val_los

In [60]:
px.imshow(x_test[0], binary_string=True, height=300, width=300) 

In [59]:
x_test[9].reshape(1, 28, 28, 1).shape

(1, 28, 28, 1)

In [58]:
out = model.predict(x_test[2].reshape(1, 28, 28, 1))
out.argmax()

1/1 [==============================] - 0s 18ms/step


1

In [64]:
names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [106]:
def load_prediction():
    model = load_model('model.h5')
    model.compile(loss=sparse_categorical_crossentropy, optimizer=SGD(), metrics=['accuracy'])
    return model

model = load_prediction()
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 5, 5, 64)          0         
                                                                 
 flatten_5 (Flatten)         (None, 1600)             

In [107]:
def predict(img):
    out = model.predict(img.reshape(1, 28, 28, 1))
    return names[out.argmax()]

In [108]:

for i in range(10):
    fig = px.imshow(x_test[i], binary_string=True, height=300, width=300, title=predict(x_test[i]))
    fig.show()

1/1 [==============================] - 0s 51ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 15ms/step


In [109]:
from PIL import Image
import requests

def download_image(url):
    response = requests.get(url,stream=True).raw
    img = Image.open(response)
    return img

In [110]:
def predict_url(url):
    img = download_image(url)
    img  = img.resize((28,28))
    img = img.convert('L')
    img = np.array(img)
    out = model.predict(img.reshape(1, 28, 28, 1))
    px.imshow(img, binary_string=True, height=300, width=300, title=names[out.argmax()]).show()
    return names[out.argmax()]

In [111]:
predict_url('https://www.charleskeith.com/dw/image/v2/BCWJ_PRD/on/demandware.static/-/Sites-ck-products/default/dwbea5035e/images/hi-res/2021-L6-CK1-91720010-1-05-1.jpg?sw=1152&sh=1536')

1/1 [==============================] - 0s 15ms/step


'Pullover'

In [96]:
predict_url('https://www.charleskeith.com/dw/image/v2/BCWJ_PRD/on/demandware.static/-/Sites-ck-products/default/dwbdc97fad/images/hi-res/2022-L7-SL1-91900002-01-1.jpg?sw=1152&sh=1536')

1/1 [==============================] - 0s 17ms/step


'Ankle boot'

In [97]:
predict_url('https://xcdn.next.co.uk/Common/Items/Default/Default/ItemImages/SearchAlt/224x336/T86329.jpg')

1/1 [==============================] - 0s 13ms/step


'Ankle boot'

In [98]:
predict_url('https://evilato.com/wp-content/uploads/2021/05/IMG-20210531-WA0000-1.jpg')

1/1 [==============================] - 0s 14ms/step


'Trouser'

In [99]:
predict_url('https://media.glamourmagazine.co.uk/photos/6407277ce5f7b373513b1f8f/1:1/w_1280,h_1280,c_limit/TROUSER%20SUITS%2070323%20MAIN.jpg')

1/1 [==============================] - 0s 14ms/step


'Pullover'